In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [ ]:
top100 = pd.read_csv('../data/RawYearlyTop100.csv')
top100.head()

In [ ]:
top100["Artist(s)"] = top100["Artist(s)"].str.split('featuring', expand=True)[0]
top100["Artist(s)"] = top100["Artist(s)"].str.split('and', expand=True)[0]
top100["Artist(s)"] = top100["Artist(s)"].str.split('&', expand=True)[0]
top100["Artist(s)"] = top100["Artist(s)"].str.split(',', expand=True)[0]

top100["Song"] = top100["Song"].str.split("/", expand=True)[0]
top100["Song"] = top100["Song"].str.replace("'", " ")

In [ ]:
top100["Query"] = "track:" + top100["Song"] + " artist:" + top100["Artist(s)"]
top100.head()

In [ ]:
top100.columns
test_df = pd.DataFrame(columns=["Year", "Rank", 'Song', 'Artist(s)', 'Query'])
for i, r in top100.head().iterrows():
    test_df = test_df.append(r)
test_df

In [ ]:
with open('../config.json') as file:
    app_data = json.load(file)

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials( \
                     client_id=app_data['client_id'], \
                     client_secret=app_data['client_secret']))

In [ ]:
def add_data_to_row(row, track):
    row['ID'] = track['id']
    row['Popularity'] = track['popularity']
    row['Release Date'] = track['album']['release_date']
    artists = []
    for i, artist in enumerate(track['artists']):
        artists.append(artist['id'])
    row['Artist IDs'] = artists
    row['Album ID'] = track['album']['id']
    return row

In [ ]:
count_missing = 0
new_columns = ["Year", "Rank", "Song ID", "Song", "Artist(s)", "Artist IDs", "Album ID", "Release Date", "Popularity", "Query"]
results = pd.DataFrame(columns=new_columns)
no_results = pd.DataFrame(columns = ["Year", "Rank", "Song", "Artist(s)", "Query"])

for i, row in top100.iterrows():
    search_res = sp.search(q=row['Query'], limit=5)
    tracks = search_res['tracks']['items']
    if len(tracks) == 0:
        count_missing += 1
        new_query = row["Artist(s)"] + row["Song"]
        search2_res = sp.search(q=new_query, limit=5)
        tracks2 = search2_res['tracks']['items']
        if len(tracks2) == 0:
            no_results = no_results.append(row, ignore_index=True)
        else:
            for idx, track in enumerate(tracks):
                row = add_data_to_row(row, track)
                results = results.append(row, ignore_index=True)
    else:
        for idx, track in enumerate(tracks):
            row = add_data_to_row(row, track)
            results = results.append(row, ignore_index=True)

results.to_csv('../data/search-results/all.csv', index=False)
no_results.to_csv('../data/search-results/all_missing.csv', index=False)